<center>Progetto realizzato da Elena Curti (matr. 185431)

# Crimes
</center>

## Introduzione

## Requisiti


## Operazioni iniziali
Creare un database ...

In [1]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687",  auth=("neo4j", "password1234"))
print("Connessione al database eseguita correttamente!")


Connessione al database eseguita correttamente!


## Descrizione del database

## 1. Quali sono gli agenti di polizia più importanti?


### Cypher query

In [2]:
def print_n_results(cq, n=100):
    ris = graph.run(cq)
    ris.sample_size = n
    display(ris)

cq = """ 
MATCH (o:Officer)<-[:INVESTIGATED_BY]-(c:Crime) 
WITH o, count(c) as num_invest
RETURN o.badge_no as badge_number, o.name as nome, o.surname as cognome, num_invest
ORDER BY num_invest DESC
LIMIT 10 ;
"""
print_n_results(cq)


# TODO questo si può rimuovere, usarlo solo per capire come cambiano i risultati
ris1 = [i["badge_number"] + " " + i["nome"] +" " + i["cognome"] for i in graph.run(cq).data()] 
def check_results(cq):
    res = [i["badge_number"] + " " + i["nome"] +" " + i["cognome"] for i in graph.run(cq).data()] 
    return res == ris1



badge_number,nome,cognome,num_invest
48-0216838,Madelon,DeSousa,50
04-5664884,Cloe,Ings,47
58-9758255,Kania,Notti,46
27-2227814,Worthy,Nettles,45
86-7588000,Ricca,Miskimmon,44
79-9843712,Olenolin,Klehyn,44
41-5548000,Winonah,Skynner,44
36-7678091,Simmonds,Greensall,43
34-6222792,Hollyanne,Orcott,43
73-9322216,Hedy,Vinson,43


### PageRank

In [3]:
NOME_GRAPH_OFFICERS = "officers"

if graph.run("call gds.graph.exists('"+NOME_GRAPH_OFFICERS+"')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("'+NOME_GRAPH_OFFICERS+'")')

cq = """
CALL gds.graph.project(
  '"""+NOME_GRAPH_OFFICERS+ """',    
  ['Officer', 'Crime'],   
  {INVESTIGATED_BY: {orientation: 'UNDIRECTED'} }
) """

graph.run(cq)
print("Proiezione creata correttamente!")

Proiezione creata correttamente!


In [4]:
cq=""" 
CALL gds.pageRank.stream('"""+NOME_GRAPH_OFFICERS+ """')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score AS pageRank
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, pageRank
ORDER BY pageRank DESC
LIMIT 10 ;
"""
print_n_results(cq)
check_results(cq)

badge_number,nome,cognome,pageRank
48-0216838,Madelon,DeSousa,22.602140755580333
04-5664884,Cloe,Ings,21.27718767680494
58-9758255,Kania,Notti,20.835536650546473
27-2227814,Worthy,Nettles,20.39388562428801
79-9843712,Olenolin,Klehyn,19.952234598029545
86-7588000,Ricca,Miskimmon,19.952234598029545
41-5548000,Winonah,Skynner,19.952234598029545
36-7678091,Simmonds,Greensall,19.510583571771075
34-6222792,Hollyanne,Orcott,19.510583571771075
73-9322216,Hedy,Vinson,19.510583571771075


False

### Degree centrality

In [5]:
cq=""" 
CALL gds.degree.stream('"""+NOME_GRAPH_OFFICERS+ """')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, score
ORDER BY score DESC
LIMIT 10 ;
"""
print_n_results(cq)

badge_number,nome,cognome,score
48-0216838,Madelon,DeSousa,50.0
04-5664884,Cloe,Ings,47.0
58-9758255,Kania,Notti,46.0
27-2227814,Worthy,Nettles,45.0
41-5548000,Winonah,Skynner,44.0
86-7588000,Ricca,Miskimmon,44.0
79-9843712,Olenolin,Klehyn,44.0
36-7678091,Simmonds,Greensall,43.0
34-6222792,Hollyanne,Orcott,43.0
73-9322216,Hedy,Vinson,43.0


### Closeness

In [6]:
cq=""" 
CALL gds.alpha.closeness.harmonic.stream('"""+NOME_GRAPH_OFFICERS+ """')
YIELD nodeId, centrality 
WITH gds.util.asNode(nodeId) AS n, centrality 
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, centrality
ORDER BY centrality DESC
LIMIT 10 ;
"""
print_n_results(cq)



badge_number,nome,cognome,centrality
48-0216838,Madelon,DeSousa,0.001680051073552636
04-5664884,Cloe,Ings,0.001579248009139478
58-9758255,Kania,Notti,0.001545646987668425
27-2227814,Worthy,Nettles,0.0015120459661973724
86-7588000,Ricca,Miskimmon,0.0014784449447263197
41-5548000,Winonah,Skynner,0.0014784449447263197
79-9843712,Olenolin,Klehyn,0.0014784449447263197
36-7678091,Simmonds,Greensall,0.001444843923255267
34-6222792,Hollyanne,Orcott,0.001444843923255267
73-9322216,Hedy,Vinson,0.001444843923255267


### Hits

In [7]:
cq=""" 
CALL gds.alpha.hits.stream('"""+NOME_GRAPH_OFFICERS+ """',  {hitsIterations: 1})
YIELD nodeId, values 
WITH gds.util.asNode(nodeId) AS n, values 
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, values
ORDER BY values DESC
LIMIT 10 ;
"""
print_n_results(cq)

badge_number,nome,cognome,values
48-0216838,Madelon,DeSousa,"{hub: 0.009576619680386747, auth: 0.05312791339882031}"
04-5664884,Cloe,Ings,"{hub: 0.009002022499563543, auth: 0.04994023859489109}"
58-9758255,Kania,Notti,"{hub: 0.008810490105955809, auth: 0.04887768032691469}"
27-2227814,Worthy,Nettles,"{hub: 0.008618957712348075, auth: 0.04781512205893828}"
86-7588000,Ricca,Miskimmon,"{hub: 0.008427425318740338, auth: 0.046752563790961876}"
41-5548000,Winonah,Skynner,"{hub: 0.008427425318740338, auth: 0.046752563790961876}"
79-9843712,Olenolin,Klehyn,"{hub: 0.008427425318740338, auth: 0.046752563790961876}"
36-7678091,Simmonds,Greensall,"{hub: 0.008235892925132604, auth: 0.045690005522985466}"
34-6222792,Hollyanne,Orcott,"{hub: 0.008235892925132604, auth: 0.045690005522985466}"
73-9322216,Hedy,Vinson,"{hub: 0.008235892925132604, auth: 0.045690005522985466}"


In [8]:
cq=""" 
CALL gds.betweenness.stream('"""+NOME_GRAPH_OFFICERS+ """')
YIELD nodeId, score 
WITH gds.util.asNode(nodeId) AS n, score 
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, score
ORDER BY score DESC
LIMIT 10 ;
"""
print_n_results(cq)


badge_number,nome,cognome,score
48-0216838,Madelon,DeSousa,1225.0
04-5664884,Cloe,Ings,1081.0
58-9758255,Kania,Notti,1035.0
27-2227814,Worthy,Nettles,990.0
41-5548000,Winonah,Skynner,946.0
86-7588000,Ricca,Miskimmon,946.0
79-9843712,Olenolin,Klehyn,946.0
36-7678091,Simmonds,Greensall,903.0
34-6222792,Hollyanne,Orcott,903.0
73-9322216,Hedy,Vinson,903.0


## 2. Trovare i gruppi di criminali

In [23]:
## Creo il project
NOME_GRAPH_CRIMINALS = "criminals"
includi_conoscenti = False
if graph.run("call gds.graph.exists('"+NOME_GRAPH_CRIMINALS+"')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("'+NOME_GRAPH_CRIMINALS+'")')

cq = """ 
  CALL gds.graph.project.cypher('""" + NOME_GRAPH_CRIMINALS+ """',
  ' MATCH (p:Person) 
    WHERE 
        EXISTS ( (p)-[:PARTY_TO]->(:Crime) )                    // Persone che hanno preso parte a un crimine
        """ +  ( "" if includi_conoscenti else "//") + """ OR EXISTS ( (p)-[]-(:Person)-[:PARTY_TO]->(:Crime) )  // Conoscenti dei criminali
    RETURN id(p) as id',
  
  'MATCH (criminal)-[]-(conoscente:Person)
    RETURN DISTINCT id(conoscente) as target,
    CASE
        WHEN EXISTS ( (conoscente)-[:KNOWS_LW]-(criminal) ) THEN 10
        WHEN EXISTS ( (conoscente)-[:FAMILY_REL]-(criminal) ) THEN 8
        // WHEN EXISTS ( (conoscente)-[:KNOWS]-(criminal) ) THEN 5
        WHEN EXISTS ( (conoscente)-[:KNOWS_PHONE]-(criminal) ) THEN 4
        WHEN EXISTS ( (conoscente)-[:KNOWS_SN]-(criminal) ) THEN 3
    END AS peso, id(criminal) as source',
  {validateRelationships:false}
); 
"""
graph.run(cq)
print("Proiezione crata correttamente!")

Proiezione crata correttamente!


### Memory estimation

In [10]:
graph.run(""" 
CALL gds.labelPropagation.write.estimate('""" + NOME_GRAPH_CRIMINALS+ """', { writeProperty: 'community' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "labelPropagation" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

UNION ALL

CALL gds.louvain.write.estimate('""" + NOME_GRAPH_CRIMINALS+ """', { writeProperty: 'community' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "louvain" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

""" )


algoritmo,nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
labelPropagation,29,30,2288,5872,[2288 Bytes ... 5872 Bytes]
louvain,29,30,7153,566272,[7153 Bytes ... 553 KiB]


### Algoritmo di Louvain

In [26]:

def print_in_columns(my_list):
    columns = 2  #if len(my_list)>10 else 3
    len_max = str(max([len(i) for i in my_list]) + 4)
    for i in range(0, len(my_list), columns):
        print("  ", end="")
        [eval('print(f"{i:'+len_max+'}", end="")') for i in my_list[i:i+columns]]
        print("")

def print_id_gruppo(query_results):
  for (communityId, gruppo) in query_results:
      print("---------- Gruppo "+str(communityId)+" ----------")
      # print(gruppo)
      print_in_columns(gruppo)
      print("")


In [28]:
cq = """ 
CALL gds.louvain.stream('""" + NOME_GRAPH_CRIMINALS + """', {relationshipWeightProperty: 'peso' })
  YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) AS node, communityId
WITH communityId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 28 ----------
  Diana Murray (900-41-3309)          Jessica Kelly (311-75-6483)         
  Kathleen Peters (250-75-5238)       Phillip Williamson (337-28-4424)    
  Raymond Walker (879-22-8665)        

---------- Gruppo 17 ----------
  Alan Ward (881-20-2396)        Brian Morales (335-71-7747)    
  Jack Powell (249-54-6589)      

---------- Gruppo 24 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 25 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      



### Modularity Optimization

In [13]:
cq = """ 
CALL gds.beta.modularityOptimization.stream('""" + NOME_GRAPH_CRIMINALS + """', {relationshipWeightProperty: 'peso' })
  YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) AS node, communityId
WITH communityId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 14 ----------
  Diana Murray (900-41-3309)       Kathleen Peters (250-75-5238)    
  Raymond Walker (879-22-8665)     

---------- Gruppo 17 ----------
  Alan Ward (881-20-2396)        Brian Morales (335-71-7747)    
  Jack Powell (249-54-6589)      

---------- Gruppo 24 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 25 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      

---------- Gruppo 28 ----------
  Jessica Kelly (311-75-6483)         Phillip Williamson (337-28-4424)    



### Weakly Connected Components

In [20]:
cq = """ 
CALL gds.wcc.stream('""" + NOME_GRAPH_CRIMINALS + """', {relationshipWeightProperty: 'peso' })
  YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS node, componentId
WITH componentId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 13 ----------
  Alan Ward (881-20-2396)             Brian Morales (335-71-7747)         
  Diana Murray (900-41-3309)          Jack Powell (249-54-6589)           
  Jessica Kelly (311-75-6483)         Kathleen Peters (250-75-5238)       
  Phillip Williamson (337-28-4424)    Raymond Walker (879-22-8665)        

---------- Gruppo 19 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 20 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      



### Strongly Connected Components 


In [24]:
cq = """ 
CALL gds.alpha.scc.stream('""" + NOME_GRAPH_CRIMINALS + """')
  YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS node, componentId
WITH componentId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 13 ----------
  Alan Ward (881-20-2396)             Brian Morales (335-71-7747)         
  Diana Murray (900-41-3309)          Jack Powell (249-54-6589)           
  Jessica Kelly (311-75-6483)         Kathleen Peters (250-75-5238)       
  Phillip Williamson (337-28-4424)    Raymond Walker (879-22-8665)        

---------- Gruppo 19 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 20 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      



### Conclusioni
...

## 3. Shortest path tra persone con precedenti di spaccio o possesso di armi
Fare lo shortest path per cercare possibili connessioni e raggruppare in base alla posizione geografica

In [15]:
NOME_GRAPH_SHORTEST_PATH = "criminals2"
if graph.run("call gds.graph.exists('"+NOME_GRAPH_SHORTEST_PATH+"')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("'+NOME_GRAPH_SHORTEST_PATH+'")')


# graph.run(""" 
# CALL gds.graph.project.cypher(
#     '""" +NOME_GRAPH_SHORTEST_PATH+"""',
#     'MATCH (p:Person) RETURN id(p) as id', 
#     '
#       MATCH p=(root)-[]->(friend_or_home)-[*0..2]->(loc:Location)
#       WHERE (friend_or_home:Person OR friend_or_home:Location) AND id(root)=2
#       RETURN p
#     ')
# """)

In [16]:
graph.run(""" 
MATCH p=(root)-[]->(friend_or_home)-[*0..2]->(loc:Location)
      WHERE (friend_or_home:Person OR friend_or_home:Location) AND id(root)=2
      RETURN p

""")

p
(Benjamin)-[:KNOWS {}]->(Frances)-[:CURRENT_ADDRESS {}]->(_753)
(Benjamin)-[:KNOWS_PHONE {}]->(Frances)-[:CURRENT_ADDRESS {}]->(_753)
(Benjamin)-[:KNOWS_SN {}]->(Brandon)-[:CURRENT_ADDRESS {}]->(_854)


In [17]:
graph.run (""" 
//  crimini senza colpevole avvenuti in luoghi in cui altri crimini vennero commessi
match (loc:Location)<--(c1:Crime)<--(p:Person), (loc)<--(c2:Crime)
where c2<>c1 AND NOT EXISTS ( (c2)<--(:Person))
 return c1, p, loc, c2
""")

c1,p,loc,c2
"(_1332:Crime {date: '31/08/2017', id: '04b0d2f1b563a455d55b2f2c24ef86564f1146630fc7d392e657ecf5f47303da', last_outcome: 'Investigation complete; no suspect identified', type: 'Robbery'})","(_214:Person {name: 'Rebecca', nhs_no: '785-79-1645', surname: 'Long'})","(_2840:Location {address: '173 Park Hill Street', latitude: 53.582779, longitude: -2.443191, postcode: 'BL1 4AR'})","(_32724:Crime {date: '11/08/2017', id: '0739499f54f4870d877750641425068e1faadf1ca3326390cd6090f539cb5f7e', last_outcome: 'Unable to prosecute suspect', type: 'Violence and sexual offences'})"
"(_1332:Crime {date: '31/08/2017', id: '04b0d2f1b563a455d55b2f2c24ef86564f1146630fc7d392e657ecf5f47303da', last_outcome: 'Investigation complete; no suspect identified', type: 'Robbery'})","(_214:Person {name: 'Rebecca', nhs_no: '785-79-1645', surname: 'Long'})","(_2840:Location {address: '173 Park Hill Street', latitude: 53.582779, longitude: -2.443191, postcode: 'BL1 4AR'})","(_32705:Crime {date: '26/08/2017', id: '641be159fa3babeb0bf1e29d66ffabe6c394e6035f55aaee004f4b827c57d316', last_outcome: 'Investigation complete; no suspect identified', type: 'Criminal damage and arson'})"
"(_1329:Crime {date: '19/08/2017', id: '34f309ebafbd1c46b5c4832f63321a10330835a5ae323952419439b69633bb2c', last_outcome: 'Unable to prosecute suspect', type: 'Public order'})","(_630:Person {name: 'Amanda', nhs_no: '455-19-0708', surname: 'Robertson'})","(_2838:Location {address: '139 Back Russell Close', latitude: 53.582117, longitude: -2.447263, postcode: 'BL1 4BA'})","(_32702:Crime {date: '9/08/2017', id: 'ce587a21f0961ba1c829efae6ef35dc746fa689f6df0a1c1074d138e8df7a2e6', last_outcome: 'Under investigation', type: 'Burglary'})"
